In [7]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

ru_url = 'kv22?xjazyk=CZ&xid=1&xv=11'

In [68]:
def get_url(specific_url):
    base_url = 'https://www.volby.cz/pls/kv2022/'
    response = requests.get(base_url + specific_url)
    soup = bs(response.content)
    return response, soup

def read_all_tables(response):
    return pd.read_html(response.text)

def get_ru_names(soup):
    return [n.text for n in soup.find_all('h3', class_='kraj_ciselnik')]

def get_ru_hrefs(soup):
    hrefs = [t.find_all('a', href=True) for t in soup.find_all('table', class_='table')]
    urls = [list(map(lambda x: x['href'], h)) for h in hrefs ]
    return urls

def scrape_regional_unit():
    ru_response, ru_soup = get_url(ru_url)
    ru_tables = read_all_tables(ru_response)
    ru_names = get_ru_names(ru_soup)
    ru_city_url = get_ru_hrefs(ru_soup)
    
    for ru_t, ru_n, ru_cu in zip(ru_tables, ru_names, ru_city_url):
        ru_t.columns = ru_t.columns.droplevel(0)
        ru_t.drop('Výběrobce', inplace=True, axis=1)
        ru_t.columns = ['ru_code', 'city_name']
        ru_t.set_index('ru_code', inplace=True, drop=True)
        ru_t['ru_name'] = ru_n
        ru_t['city_url'] = ru_cu
    
    return pd.concat(ru_tables)

In [69]:
ru_table_all = scrape_regional_unit()
ru_table_all.head()

,city_name,ru_name,city_url
ru_code,,,
CZ0100,Praha,Hlavní město Praha,kv222?xjazyk=CZ&xid=1&xv=11&xnumnuts=1100
CZ0201,Benešov,Středočeský kraj,kv222?xjazyk=CZ&xid=1&xv=11&xnumnuts=2101
CZ0202,Beroun,Středočeský kraj,kv222?xjazyk=CZ&xid=1&xv=11&xnumnuts=2102
CZ0203,Kladno,Středočeský kraj,kv222?xjazyk=CZ&xid=1&xv=11&xnumnuts=2103
CZ0204,Kolín,Středočeský kraj,kv222?xjazyk=CZ&xid=1&xv=11&xnumnuts=2104
